### 7. Collapsed Gibbs Sampling

<img src='attachment/69. LDA生成文本过程.png' style='zoom:50%'/>

常规的Gibbs Sampling过程为:


$$
\begin{array}{l}
for \quad i=1,2,\cdots,N \\
\quad \quad \theta_{i} \sim P(\theta_{i}|w, \alpha, \beta, \phi, Z, \theta_{-i}) \\
\quad \\
for \quad k=1,2,\cdots,K \\
\quad \quad  \phi_{k} \sim P(\phi_{k}|w, \alpha, \beta, \theta, Z, \phi_{-k}) \\
\quad \\
for \quad i=1,2,\cdots,N \\
\quad for \quad j=1,2,\cdots,N_{i} \\
\quad \quad \quad Z_{ij} \sim P(Z_{ij}|w, \alpha, \beta, \theta, \phi, Z_{-ij})
\end{array}
$$

存在两个问题:

1. 参数多，效率低；

2. 收敛速度慢

**Collapsed Gibbs Sampling:**

对后验分布$~P(Z_{ij}|\alpha, \beta, w, Z_{-ij})$进行Gibbs抽样，得到分布$~P(Z_{ij}|\alpha, \beta, w, Z_{-ij})$的样本集合；再利用这个样本集合对参数$\theta, ~\phi$进行估计，最终得到LDA模型的所有参数估计。

$$
\begin{array}{l}
for \quad i=1,2,\cdots,N \\
\quad for \quad j=1,2,\cdots,N_{i} \\
\quad \quad \quad Z_{ij} \sim P(Z_{ij}|w, \alpha, \beta, Z_{-ij})
\end{array}
$$

#### 后验概率分布$~P(Z_{ij}|w, \alpha, \beta,Z_{-ij})$推导

$$
P(Z_{ts}|Z_{-ts}, \alpha, \beta, w) = \frac{P(Z, w|\alpha, \beta)}{P(Z_{-ts}, w|\alpha, \beta)} \tag{1}
$$

对于分子部分:

$$
P(Z, w|\alpha, \beta) = \underset{A}{\underbrace{\int P(Z|\theta)P(\theta|\alpha)\mathrm{~d}\theta}} \cdot \underset{B}{\underbrace{\int P(w|Z, \phi)P(\phi|\beta)\mathrm{~d}\phi}} \tag{2}
$$

$$
\begin{eqnarray}
A &\Rightarrow& \int P(Z|\theta)\ P(\theta|\alpha)\mathrm{~d}\theta \\
&=& \int \prod_{i=1}^{N} \prod_{j=1}^{N_{i}} \underset{\color{red}{\text{~Mult}}}{P(Z_{ij}|\theta_{i})} \cdot \prod_{i=1}^{N} \underset{\color{red}{\text{~Dir}}}{P(\theta_{i} | \alpha)} \mathrm{~d}\theta \\
\quad \\
&=& \int \prod_{i=1}^{N} \prod_{j=1}^{N_{i}}\prod_{k=1}^{K}\theta_{ik}^{I(Z_{ij}~ =k)} \cdot \prod_{i=1}^{N} \frac{1}{B(\alpha)} \prod_{k=1}^{K} \theta_{ik}^{\alpha_{k}-1}\mathrm{~d}\theta_{i} \\
\quad \\
&=& \prod_{i=1}^{N} \frac{1}{B(\alpha)} \int \prod_{j=1}^{N_{i}}\prod_{k=1}^{K}\theta_{ik}^{I(Z_{ij}~ =k)} \cdot \prod_{k=1}^{K} \theta_{ik}^{\alpha_{k}-1}\mathrm{~d}\theta_{i} \\
\quad \\
&=& \prod_{i=1}^{N} \frac{1}{B(\alpha)} \int \prod_{k=1}^{K}\theta_{ik}^{\sum_{j=1}^{N_{~i}}I(Z_{ij}~ =k) + \alpha_{k}-1} \mathrm{~d}\theta_{i} \\
\quad \\
&=& \prod_{i=1}^{N} \frac{1}{B(\alpha)} \cdot B\left( \alpha + \sum_{j=1}^{N_{i}}I_{k}(Z_{ij}) \right) \tag{3}
\end{eqnarray}
$$

$$
\begin{eqnarray}
B &\Rightarrow& \int \underset{\color{red}{\text{~Mult}}}{P(w|Z, \phi)} \cdot \underset{\color{red}{\text{~Dir}}}{P(\phi \mid\beta)}\mathrm{~d}\phi \\
\quad \\
&=& \int \prod_{k=1}^{K}~ \prod_{i:Z_{i}=k}~ \prod_{v=1}^{|V|} \phi_{kv}^{I(w_{i}=v)} \cdot \frac{1}{B(\beta)}\prod_{k=1}^{K}\prod_{v=1}^{|V|}\phi_{kv}^{\beta_{v}-1}\mathrm{~d}\phi_{k} \\
\quad \\
&=& \prod_{k=1}^{K} \frac{1}{B(\beta)} \int \prod_{v=1}^{|V|} \phi_{kv}^{\sum_{i:Z_{~i~}=k~}I(w_{i}=v) + \beta_{v}-1}\mathrm{~d}\phi_{k} \\
\quad \\
&=& \prod_{k=1}^{K} \frac{1}{B(\beta)} \cdot B\left( \sum_{i:Z_{i}=k~}\mathrm{I}_{v}(w_{i}) + \beta \right) \tag{4}
\end{eqnarray}
$$

由（2）~（4）式可得: 

$$
P(Z, w|\alpha, \beta) = \prod_{i=1}^{N} \frac{1}{B(\alpha)}  B\left( \alpha + \sum_{j=1}^{N_{i}}I_{k}(Z_{ij}) \right) \cdot \prod_{k=1}^{K} \frac{1}{B(\beta)}  B\left( \sum_{i:Z_{i}=k~}\mathrm{I}_{v}(w_{i}) + \beta \right) \tag{5}
$$

同理，可得到式（2）的分母部分: 

$$
P(Z_{-ts}, w \mid \alpha, \beta) = \prod_{i=1}^{N} \frac{1}{B(\alpha)}  B\left( \alpha + \sum_{j=1;\delta_{j}\neq(t,s)}^{N_{i}}I_{k}(Z_{ij}) \right) \cdot \prod_{k=1}^{K} \frac{1}{B(\beta)}  B\left( \sum_{i:Z_{i}=k;\delta_{i}\neq (t,s)~}\mathrm{I}_{v}(w_{i}) + \beta \right) \tag{6}
$$

由（5）（6）可得到后验概率 $P(Z_{ts}|\alpha,\beta,w,Z_{-ts})$:

$$
P(Z_{ts}|\alpha,\beta,w,Z_{-ts}) = \frac{\alpha_{k} + n_{dk}}{\sum_{k=1}^{K}\alpha_{k}+n_{d}} \cdot \frac{\beta_{w_{i}} + n_{k,w_{i}}}{\sum_{v=1}^{|V|}\beta_{v} + n_{k}} \tag{7}
$$

其中: 

$n_{d}:$ 文档$~d~$中的单词个数 

$n_{dk}:$ 文档$~d~$中属于主题$~k~$的个数

$n_{k}:$ 所有文档中，属于主题$~k~$的单词个数

$n_{k,w_{i}}:$ 单词$~w_{i}~$有多少次被属于主题$~k~$(<font color=blue>但排除当前单词</font>)

#### $Z_{ij}~$迭代采样过程

$$
\begin{array}{l}
\text{for} \ \text{iterations}=1,2,\cdots \\
\quad \text{for} \ i=1,2,\cdots,N \\
\quad \quad \text{for} \ i=1,2,\cdots,N_{i} \\
\quad \quad \quad  Z_{ij} = \text{sample from}~ P(Z_{ij}|w, \alpha, \beta, Z_{-ij}) \\
\quad \{Z_{ij}\} \rightarrow \phi,\theta
\end{array}
$$

<img src='attachment/69. Sample Zij.png' style='zoom:50%'/>

$$
P(Z_{ts}=k \mid \alpha,\beta,w,Z_{-ts}) = \frac{\alpha_{k} + n_{dk}}{\sum_{k=1}^{K}\alpha_{k}+n_{d}} \cdot \frac{\beta_{w_{i}} + n_{k,w_{i}}}{\sum_{v=1}^{|V|}\beta_{v} + n_{k}}(\color{red}{exclude ~ current ~ word})
$$

vocab={今天，我们，上课，天气}

#### Step1: 初始化

$\alpha=(0.1,~ 0.1,~ 0.1) \quad \beta=(0.01, ~0.01, ~0.01, ~0.01) \quad Z$如图

#### Step2: Sample $Z_{11}$

$Z_{11}$（“今天”）的主题(<font color=blue>但排除当前单词“今天”</font>)

$$
P(Z_{11}=1|\alpha, \beta, w, Z_{-11})=\frac{0.1 + 0}{0.3 + 3} \cdot \frac{0.01 + 0}{0.04 + 2} = \frac{0.1*0.01}{3.3 * 2.04} \\
P(Z_{11}=2|\alpha, \beta, w, Z_{-11})=\frac{0.1 + 2}{0.3 + 3} \cdot \frac{0.01 + 1}{0.04 + 2} = \frac{2.1*1.01}{3.3 * 2.04} \\
P(Z_{11}=3|\alpha, \beta, w, Z_{-11})=\frac{0.1 + 1}{0.3 + 3} \cdot \frac{0.01 + 0}{0.04 + 2} = \frac{1.1*0.01}{3.3 * 2.04} \\
$$

假设从Mult分布中采样得到 $Z_{11}=2$，更新“今天”的主题为2。

同理，$Z_{12}$（“我们”）的主题

$$
P(Z_{12}=1|\alpha, \beta, w, Z_{-12})=\frac{0.1 + 0}{0.3 + 3} \cdot \frac{0.01 + 1}{0.04 + 2} = \frac{0.1*1.01}{3.3 * 2.04} \\
P(Z_{12}=2|\alpha, \beta, w, Z_{-12})=\frac{0.1 + 2}{0.3 + 3} \cdot \frac{0.01 + 1}{0.04 + 2} = \frac{2.1*1.01}{3.3 * 2.04} \\
P(Z_{12}=3|\alpha, \beta, w, Z_{-12})=\frac{0.1 + 1}{0.3 + 3} \cdot \frac{0.01 + 0}{0.04 + 2} = \frac{1.1*0.01}{3.3 * 2.04} \\
$$

假设从Mult分布中采样得到 $Z_{12}=2$